In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coffee-sales/index_2.csv
/kaggle/input/coffee-sales/index_1.csv


In [21]:
# !pip install -q google-generativeai

## Project Overview

This capstone project builds an AI-powered Enterprise Agent that analyzes uploaded business data (e.g., sales reports), calculates KPIs, and generates natural language summaries and visual dashboards to support decision-making.

### Key Features:
- Upload CSV data
- Analyze KPIs (e.g., revenue, top products)
- Generate summaries using Gemini
- Display results in a glowing dashboard

## Step 1: API Conection with Google AI Studio 

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


## Step 2: ADK Components Importing

In [4]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Step 3: Helper functions defined

In [6]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [7]:
root_agent = Agent(
    name="helpful_assistant",
    model="gemini-2.5-flash-lite",
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


## 🤖 Step 4:Agent Architecture

### Flow
User uploads CSV → Agent analyzes KPIs → Gemini generates summary → Dashboard displays results

### Agents
- **Agent 1: Data Loader** → handles CSV upload and cleaning
- **Agent 2: KPI Analyzer** → calculates metrics 
- **Agent 3: Summary Generator** → uses Gemini to explain KPIs in natural language
- **Agent 4: Dashboard Renderer** → visualizes charts and summaries

### Tools
- `upload_csv()` → load and parse data
- `analyze_kpis()` → compute KPIs with Pandas
- `summarize_with_gemini()` → generate insights
- `generate_charts()` → create Plotly visualizations
- `render_dashboard()` → display results in notebook

In [8]:
import pandas as pd
import plotly.express as px

# Tool 1: Upload CSV
def upload_csv(file_path):

    try:
        df = pd.read_csv(file_path)
        print(" Data uploaded successfully")
        return df
    except Exception as e:
        print(f" Error uploading file: {e}")
        return None

# Tool 2: Analyze KPIs
def analyze_kpis(df):
    
    if df is None:
        return "No data available"
    
    kpis = {
        "Total Rows": len(df),
        "Columns": list(df.columns),
        "Missing Values": df.isnull().sum().to_dict(),
        "Data Types": df.dtypes.astype(str).to_dict()
    }
    print(" KPI Analysis Complete")
    return kpis

# Tool 3: Summarize with Gemini (placeholder)
def summarize_with_gemini(text, llm):
  
    response = llm.invoke(text)
    return response.content

# Tool 4: Generate Charts
def generate_charts(df, column):
    
    try:
        fig = px.histogram(df, x=column, title=f"Distribution of {column}")
        fig.show()
    except Exception as e:
        print(f" Chart generation failed: {e}")

In [9]:
# Load your dataset
file_path = "/kaggle/input/coffee-sales/index_2.csv"
data = upload_csv(file_path)

# Analyze KPIs
kpis = analyze_kpis(data)
print(kpis)

generate_charts(data, column="coffee_name")  


 Data uploaded successfully
 KPI Analysis Complete
{'Total Rows': 262, 'Columns': ['date', 'datetime', 'cash_type', 'money', 'coffee_name'], 'Missing Values': {'date': 0, 'datetime': 0, 'cash_type': 0, 'money': 0, 'coffee_name': 0}, 'Data Types': {'date': 'object', 'datetime': 'object', 'cash_type': 'object', 'money': 'float64', 'coffee_name': 'object'}}


## 🤖 Step 6: Multi-Agent System Mapping

This project demonstrates a **sequential multi-agent system** using custom tools and Gemini (LLM-powered).

###  Agents in the System
1. **Agent 1: Data Loader**  
   - Loads the dataset from CSV using `upload_csv()`  

2. **Agent 2: KPI Analyzer**  
   - Calculates basic dataset metrics (rows, columns, missing values, data types) using `analyze_kpis()`  

3. **Agent 3: Summary Generator**  
   - Uses Gemini (`google-generativeai`) to generate a human-readable summary of the KPIs  

4. **Agent 4: Chart Generator**  
   - Visualizes dataset columns with Plotly using `generate_charts()`  


##  What KPI Means

**KPI = Key Performance Indicator**

In this project, KPIs are **basic metrics about the dataset** that help quickly understand its structure and quality.

### KPIs in Our Tool
- **Total Rows** → Number of records (transactions) in the dataset  
- **Columns** → Names of all the fields in the dataset  
- **Missing Values** → Count of empty/null values in each column  
- **Data Types** → Whether each column is numeric, text, or date/time  

---

These KPIs form the **foundation for deeper analysis and visualization**, ensuring the dataset is clean and ready for exploration.

In [10]:
# Agent 1: Data Loader
def agent_data_loader(file_path):
    print(" Agent 1: Loading data...")
    df = upload_csv(file_path)
    return df

# Agent 2: KPI Analyzer
def agent_kpi_analyzer(df):
    print(" Agent 2: Analyzing KPIs...")
    kpis = analyze_kpis(df)
    print(kpis)
    return kpis

# Agent 3: Summary Generator
def agent_summary_generator(kpis, llm):
    print(" Agent 3: Generating summary with Gemini...")
    summary_text = f"Here are the KPIs: {kpis}"
    summary = summarize_with_gemini(summary_text, llm)
    print(summary)
    return summary

# Agent 4: Chart Generator
def agent_chart_generator(df, column):
    print(" Agent 4: Generating chart...")
    generate_charts(df, column)

In [11]:
import google.generativeai as genai

api_key =os.environ.get("GOOGLE API KEY")
genai.configure(api_key=api_key)
# Initialize Gemini model
model = genai.GenerativeModel("gemini-2.5-flash-lite")

In [12]:
# Tool 3: Summarize with Gemini (using google-generativeai)
def summarize_with_gemini(text, model):
    response = model.generate_content(text)
    return response.text

In [13]:
file_path = "/kaggle/input/coffee-sales/index_2.csv"

df = agent_data_loader(file_path)
kpis = agent_kpi_analyzer(df)
summary = agent_summary_generator(kpis, model)   
agent_chart_generator(df, column="coffee_name")  

 Agent 1: Loading data...
 Data uploaded successfully
 Agent 2: Analyzing KPIs...
 KPI Analysis Complete
{'Total Rows': 262, 'Columns': ['date', 'datetime', 'cash_type', 'money', 'coffee_name'], 'Missing Values': {'date': 0, 'datetime': 0, 'cash_type': 0, 'money': 0, 'coffee_name': 0}, 'Data Types': {'date': 'object', 'datetime': 'object', 'cash_type': 'object', 'money': 'float64', 'coffee_name': 'object'}}
 Agent 3: Generating summary with Gemini...
Here's a summary of the KPIs provided, along with some interpretations and potential next steps:

**Key Performance Indicators (KPIs) Summary:**

*   **Total Rows:** 262 - This indicates the total number of records or observations in your dataset.
*   **Columns:**
    *   `date`
    *   `datetime`
    *   `cash_type`
    *   `money`
    *   `coffee_name`
    This lists the names of all the features or variables in your dataset.
*   **Missing Values:**
    *   `date`: 0
    *   `datetime`: 0
    *   `cash_type`: 0
    *   `money`: 0
    *  

## 🛠️ Step 6: Observability, Memory & Advanced Agent Features

In this step, we extend our multi-agent system by adding **observability** and **memory/state management**.  
This ensures our agents are not only functional but also **traceable, explainable, and persistent**.

###  Goals of Step 6
1. **Observability**
   - Add logging for each agent’s actions (start, success, error).
   - Enable tracing of agent workflows.
   - Collect simple metrics (e.g., execution time, number of rows processed).

2. **Sessions & Memory**
   - Introduce session/state management so agents can remember context across runs.
   - Demonstrate long-term memory (e.g., storing dataset KPIs for reuse).
   - Show how memory can be compacted or reset when needed.


###  Why This Matters
- **Observability** makes your agents transparent and debuggable. You can prove what happened at each step.  
- **Memory** allows agents to build on past runs, making them smarter and more useful over time.  
- Together, these features elevate your project from a simple pipeline to a **professional-grade agent system** ready for deployment.

In [14]:
import time


memory_store = {}


def log_event(agent_name, event, details=""):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {agent_name} - {event} {details}")

def agent_data_loader(file_path):
    log_event("Agent 1", "START", f"Loading data from {file_path}")
    df = upload_csv(file_path)
    log_event("Agent 1", "SUCCESS", f"Data uploaded with {len(df)} rows")
    return df

def agent_kpi_analyzer(df):
    log_event("Agent 2", "START", "Analyzing KPIs")
    kpis = analyze_kpis(df)
    log_event("Agent 2", "SUCCESS", "KPI Analysis Complete")
    # Save KPIs to memory for reuse
    memory_store["last_kpis"] = kpis
    return kpis

def agent_summary_generator(kpis, model):
    log_event("Agent 3", "START", "Generating summary with Gemini")
    summary_text = f"Here are the KPIs: {kpis}"
    summary = summarize_with_gemini(summary_text, model)
    log_event("Agent 3", "SUCCESS", "Summary generated")
    memory_store["last_summary"] = summary
    return summary

def agent_chart_generator(df, column):
    log_event("Agent 4", "START", f"Generating chart for column: {column}")
    generate_charts(df, column)
    log_event("Agent 4", "SUCCESS", f"Chart generated for {column}")

In [15]:
file_path = "/kaggle/input/coffee-sales/index_2.csv"

df = agent_data_loader(file_path)
kpis = agent_kpi_analyzer(df)
summary = agent_summary_generator(kpis, model)
agent_chart_generator(df, column="coffee_name")

print("\n--- Memory Store ---")
print(memory_store)

[2025-11-25 13:57:42] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:57:42] Agent 1 - SUCCESS Data uploaded with 262 rows
[2025-11-25 13:57:42] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:57:42] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:57:42] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:57:47] Agent 3 - SUCCESS Summary generated
[2025-11-25 13:57:47] Agent 4 - START Generating chart for column: coffee_name


[2025-11-25 13:57:47] Agent 4 - SUCCESS Chart generated for coffee_name

--- Memory Store ---
{'last_kpis': {'Total Rows': 262, 'Columns': ['date', 'datetime', 'cash_type', 'money', 'coffee_name'], 'Missing Values': {'date': 0, 'datetime': 0, 'cash_type': 0, 'money': 0, 'coffee_name': 0}, 'Data Types': {'date': 'object', 'datetime': 'object', 'cash_type': 'object', 'money': 'float64', 'coffee_name': 'object'}}, 'last_summary': 'Here\'s a breakdown of the KPIs you provided, along with some interpretations and potential next steps:\n\n**Key Performance Indicators (KPIs) Overview:**\n\n*   **Total Rows:** 262\n    *   This tells you the total number of records (transactions, in this case) in your dataset. It\'s a fundamental measure of the dataset\'s size.\n*   **Columns:** [\'date\', \'datetime\', \'cash_type\', \'money\', \'coffee_name\']\n    *   These are the attributes or features available in your data. They describe each transaction.\n*   **Missing Values:**\n    *   **date:** 0 mi

## 🧪 Step 7: Agent Evaluation

In this step, we focus on **evaluating our agents** to ensure they are reliable, accurate, and efficient.  
Evaluation helps us identify strengths, weaknesses, and opportunities for improvement in the multi-agent system.

###  Goals of Step 7
1. **Functional Testing**
   - Verify that each agent performs its intended task (data loading, KPI analysis, summary generation, charting).
   - Check error handling (e.g., invalid file paths, missing columns).

2. **Performance Metrics**
   - Measure execution time for each agent.
   - Track resource usage (rows processed, memory consumed).
   - Compare sequential vs. parallel execution efficiency (if applicable).

3. **Quality Assessment**
   - Evaluate the clarity and usefulness of Gemini-generated summaries.
   - Validate KPI accuracy against the dataset.
   - Ensure charts are meaningful and correctly represent the data.


###  Why This Matters
- **Evaluation ensures trustworthiness**: You can demonstrate that your agents consistently produce correct results.  
- **Performance insights**: Helps optimize workflows for speed and efficiency.  
- **Quality control**: Guarantees that outputs (summaries, charts, KPIs) are accurate and useful for decision-making.

In [16]:
import time

def timed_execution(agent_func, *args, **kwargs):
    start = time.time()
    try:
        result = agent_func(*args, **kwargs)
        end = time.time()
        duration = round(end - start, 3)
        log_event(agent_func.__name__, "SUCCESS", f"Execution time: {duration} seconds")
        return result, duration
    except Exception as e:
        end = time.time()
        duration = round(end - start, 3)
        log_event(agent_func.__name__, "ERROR", f"{str(e)} | Execution time: {duration} seconds")
        return None, duration

def evaluate_agents(file_path, model):
    print("\n=== Agent Evaluation Started ===")

   
    df, t1 = timed_execution(agent_data_loader, file_path)

    kpis, t2 = timed_execution(agent_kpi_analyzer, df)

    summary, t3 = timed_execution(agent_summary_generator, kpis, model)

    _, t4 = timed_execution(agent_chart_generator, df, "cash_type")

    print("\n=== Evaluation Results ===")
    print(f"Agent 1 (Data Loader): {t1} seconds")
    print(f"Agent 2 (KPI Analyzer): {t2} seconds")
    print(f"Agent 3 (Summary Generator): {t3} seconds")
    print(f"Agent 4 (Chart Generator): {t4} seconds")

    print("\n--- Memory Store ---")
    print(memory_store)

    print("\n=== Agent Evaluation Complete ===")

In [17]:
file_path = "/kaggle/input/coffee-sales/index_2.csv"
evaluate_agents(file_path, model)


=== Agent Evaluation Started ===
[2025-11-25 13:57:47] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:57:47] Agent 1 - SUCCESS Data uploaded with 262 rows
[2025-11-25 13:57:47] agent_data_loader - SUCCESS Execution time: 0.004 seconds
[2025-11-25 13:57:47] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:57:47] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:57:47] agent_kpi_analyzer - SUCCESS Execution time: 0.002 seconds
[2025-11-25 13:57:47] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:57:51] Agent 3 - SUCCESS Summary generated
[2025-11-25 13:57:51] agent_summary_generator - SUCCESS Execution time: 4.677 seconds
[2025-11-25 13:57:51] Agent 4 - START Generating chart for column: cash_type


[2025-11-25 13:57:51] Agent 4 - SUCCESS Chart generated for cash_type
[2025-11-25 13:57:51] agent_chart_generator - SUCCESS Execution time: 0.047 seconds

=== Evaluation Results ===
Agent 1 (Data Loader): 0.004 seconds
Agent 2 (KPI Analyzer): 0.002 seconds
Agent 3 (Summary Generator): 4.677 seconds
Agent 4 (Chart Generator): 0.047 seconds

--- Memory Store ---
{'last_kpis': {'Total Rows': 262, 'Columns': ['date', 'datetime', 'cash_type', 'money', 'coffee_name'], 'Missing Values': {'date': 0, 'datetime': 0, 'cash_type': 0, 'money': 0, 'coffee_name': 0}, 'Data Types': {'date': 'object', 'datetime': 'object', 'cash_type': 'object', 'money': 'float64', 'coffee_name': 'object'}}, 'last_summary': 'Okay, you\'ve provided the Key Performance Indicators (KPIs) for your dataset. Let\'s break down what this tells us and what it implies.\n\n**Summary of your KPIs:**\n\n*   **Total Rows:** 262 (This is a relatively small dataset.)\n*   **Columns:** You have five columns: `date`, `datetime`, `cash_t

## 🔄 Step 8: A2A Protocol (Agent-to-Agent Communication)

In this step, we extend our multi-agent system to demonstrate **Agent-to-Agent (A2A) communication**.  
This means agents can **pass information directly to each other**, instead of only relying on the user or a central controller.

### 🔹 Goals of Step 8
1. **Sequential A2A Communication**
   - Agent 1 (Data Loader) passes the dataset directly to Agent 2 (KPI Analyzer).
   - Agent 2 passes KPIs directly to Agent 3 (Summary Generator).
   - Agent 3 passes the summary to Agent 4 (Chart Generator) for visualization context.

2. **Parallel Agents**
   - Demonstrate agents running in parallel (e.g., KPI Analyzer and Chart Generator working at the same time on the dataset).

3. **Loop Agents**
   - Show how agents can re-run until a condition is met (e.g., keep analyzing until no missing values are found).


### 🔹 Why This Matters
- **Efficiency** → Agents can work together without waiting for user input at every step.  
- **Scalability** → Parallel agents reduce execution time.  
- **Autonomy** → Loop agents allow self-correction and iterative improvement.  
- **Professional-grade system** → Demonstrates advanced agent orchestration beyond simple sequential workflows.

In [18]:
import concurrent.futures

def sequential_a2a_pipeline(file_path, model):
    print("\n=== Sequential A2A Pipeline Started ===")

    # Agent 1 → Agent 2
    df = agent_data_loader(file_path)
    kpis = agent_kpi_analyzer(df)

    # Agent 2 → Agent 3
    summary = agent_summary_generator(kpis, model)

    # Agent 3 → Agent 4
    agent_chart_generator(df, column="cash_type")
    print("\n===  Sequential A2A Pipeline Complete ===")
    return df, kpis, summary


def parallel_a2a_pipeline(df, model):
    print("\n===  Parallel A2A Pipeline Started ===")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_kpis = executor.submit(agent_kpi_analyzer, df)
        future_chart = executor.submit(agent_chart_generator, df, "coffee_name")

        kpis = future_kpis.result()
        chart_result = future_chart.result()

    summary = agent_summary_generator(kpis, model)

    print("\n===  Parallel A2A Pipeline Complete ===")
    return kpis, summary, chart_result


def loop_agent(df):
    print("\n===  Loop Agent Started ===")
    iteration = 0
    while True:
        iteration += 1
        kpis = agent_kpi_analyzer(df)
        missing = sum(kpis["Missing Values"].values())
        print(f"Iteration {iteration}: Missing values = {missing}")
        if missing == 0:
            print("No missing values found. Loop ends.")
            break
        else:
            print("Re-analyzing until dataset is clean...")
    print("\n===  Loop Agent Complete ===")
    return kpis
def parallel_a2a_pipeline(df, model):
    print("\n===  Parallel A2A Pipeline Started ===")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_kpis = executor.submit(agent_kpi_analyzer, df)
        future_chart = executor.submit(agent_chart_generator, df, "money")

        kpis = future_kpis.result()
        chart_result = future_chart.result()

    summary = agent_summary_generator(kpis, model)

    print("\n=== Parallel A2A Pipeline Complete ===")
    return kpis, summary, chart_result


def loop_agent(df):
    print("\n===  Loop Agent Started ===")
    iteration = 0
    while True:
        iteration += 1
        kpis = agent_kpi_analyzer(df)
        missing = sum(kpis["Missing Values"].values())
        print(f"Iteration {iteration}: Missing values = {missing}")
        if missing == 0:
            print("No missing values found. Loop ends.")
            break
        else:
            print("Re-analyzing until dataset is clean...")
    print("\n=== Loop Agent Complete ===")
    return kpis

In [19]:
file_path = "/kaggle/input/coffee-sales/index_2.csv"

df, kpis, summary = sequential_a2a_pipeline(file_path, model)
kpis, summary, chart_result = parallel_a2a_pipeline(df, model)
final_kpis = loop_agent(df)


=== Sequential A2A Pipeline Started ===
[2025-11-25 13:57:51] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:57:51] Agent 1 - SUCCESS Data uploaded with 262 rows
[2025-11-25 13:57:51] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:57:51] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:57:51] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:57:56] Agent 3 - SUCCESS Summary generated
[2025-11-25 13:57:56] Agent 4 - START Generating chart for column: cash_type


[2025-11-25 13:57:56] Agent 4 - SUCCESS Chart generated for cash_type

===  Sequential A2A Pipeline Complete ===

===  Parallel A2A Pipeline Started ===
[2025-11-25 13:57:56] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:57:56] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:57:56] Agent 4 - START Generating chart for column: money


[2025-11-25 13:57:56] Agent 4 - SUCCESS Chart generated for money
[2025-11-25 13:57:56] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:58:03] Agent 3 - SUCCESS Summary generated

=== Parallel A2A Pipeline Complete ===

===  Loop Agent Started ===
[2025-11-25 13:58:03] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:58:03] Agent 2 - SUCCESS KPI Analysis Complete
Iteration 1: Missing values = 0
No missing values found. Loop ends.

=== Loop Agent Complete ===



## 🚀 Step 9: Agent Deployment

This is the final step of the capstone project, where we **deploy our multi-agent system**.  
Deployment ensures that the system is not just a prototype but can run in a **production-like environment**.

###  Goals of Step 9
1. **Deployment Pipeline**
   - Package the agents into a reusable workflow.
   - Demonstrate running the pipeline end-to-end (data loading → KPI analysis → summary → charting).

2. **Long-Running Operations**
   - Show how agents can be paused/resumed.
   - Demonstrate resilience in case of interruptions.

3. **Production Readiness**
   - Ensure logging, memory, and evaluation are integrated.
   - Provide a clean interface for running the system (single function call).


###  Why This Matters
- **Deployment makes your project complete**: It moves from an academic prototype to a professional-grade system.  
- **Reusability**: Anyone can run the pipeline without needing to understand internal details.  
- **Portfolio-ready**: Demonstrates end-to-end mastery of multi-agent systems, tools, memory, observability, evaluation, and deployment.

In [20]:
def deploy_pipeline(file_path, model, mode="sequential"):
    """
    Deploy the multi-agent system end-to-end.
    Modes:
      - "sequential" → agents run one after another
      - "parallel"   → KPI Analyzer + Chart Generator run together
      - "loop"       → KPI Analyzer re-runs until no missing values
    """

    log_event("Deployment", "START", f"Mode: {mode}")

    if mode == "sequential":
        df, kpis, summary = sequential_a2a_pipeline(file_path, model)
        result = {"df": df, "kpis": kpis, "summary": summary}

    elif mode == "parallel":
        df = agent_data_loader(file_path)
        kpis, summary, chart_result = parallel_a2a_pipeline(df, model)
        result = {"df": df, "kpis": kpis, "summary": summary, "chart": chart_result}

    elif mode == "loop":
        df = agent_data_loader(file_path)
        final_kpis = loop_agent(df)
        summary = agent_summary_generator(final_kpis, model)
        agent_chart_generator(df, "money")
        result = {"df": df, "kpis": final_kpis, "summary": summary}

    else:
        raise ValueError("Invalid mode. Choose 'sequential', 'parallel', or 'loop'.")

    log_event("Deployment", "SUCCESS", f"Pipeline completed in {mode} mode")
    return result


file_path = "/kaggle/input/coffee-sales/index_2.csv"

deploy_sequential = deploy_pipeline(file_path, model, mode="sequential")
deploy_parallel = deploy_pipeline(file_path, model, mode="parallel")
deploy_loop = deploy_pipeline(file_path, model, mode="loop")

[2025-11-25 13:58:03] Deployment - START Mode: sequential

=== Sequential A2A Pipeline Started ===
[2025-11-25 13:58:03] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:58:03] Agent 1 - SUCCESS Data uploaded with 262 rows
[2025-11-25 13:58:03] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:58:03] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:58:03] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:58:07] Agent 3 - SUCCESS Summary generated
[2025-11-25 13:58:07] Agent 4 - START Generating chart for column: cash_type


[2025-11-25 13:58:07] Agent 4 - SUCCESS Chart generated for cash_type

===  Sequential A2A Pipeline Complete ===
[2025-11-25 13:58:07] Deployment - SUCCESS Pipeline completed in sequential mode
[2025-11-25 13:58:07] Deployment - START Mode: parallel
[2025-11-25 13:58:07] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:58:07] Agent 1 - SUCCESS Data uploaded with 262 rows

===  Parallel A2A Pipeline Started ===
[2025-11-25 13:58:07] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:58:07] Agent 2 - SUCCESS KPI Analysis Complete
[2025-11-25 13:58:07] Agent 4 - START Generating chart for column: money


[2025-11-25 13:58:07] Agent 4 - SUCCESS Chart generated for money
[2025-11-25 13:58:07] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:58:15] Agent 3 - SUCCESS Summary generated

=== Parallel A2A Pipeline Complete ===
[2025-11-25 13:58:15] Deployment - SUCCESS Pipeline completed in parallel mode
[2025-11-25 13:58:15] Deployment - START Mode: loop
[2025-11-25 13:58:15] Agent 1 - START Loading data from /kaggle/input/coffee-sales/index_2.csv
 Data uploaded successfully
[2025-11-25 13:58:15] Agent 1 - SUCCESS Data uploaded with 262 rows

===  Loop Agent Started ===
[2025-11-25 13:58:15] Agent 2 - START Analyzing KPIs
 KPI Analysis Complete
[2025-11-25 13:58:15] Agent 2 - SUCCESS KPI Analysis Complete
Iteration 1: Missing values = 0
No missing values found. Loop ends.

=== Loop Agent Complete ===
[2025-11-25 13:58:15] Agent 3 - START Generating summary with Gemini
[2025-11-25 13:58:22] Agent 3 - SUCCESS Summary generated
[2025-11-25 13:58:22] Agent 4 - START Generating chart

[2025-11-25 13:58:22] Agent 4 - SUCCESS Chart generated for money
[2025-11-25 13:58:22] Deployment - SUCCESS Pipeline completed in loop mode
